In [0]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import numpy as np
import nltk 
import random 
from nltk.tokenize import word_tokenize
nltk.download('punkt') 
nltk.download('stopwords') 
#nltk.download('movie_reviews') 
nltk.download('wordnet') 

In [0]:
# Imported the necessary libraries
import csv
import urllib.request as urllib2
import matplotlib.pyplot as plt
from nltk import FreqDist
import pandas as pd


In [114]:
# Importing data using url
url = 'https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv'

# reading data using pandas and converting into dataframe
data = pd.read_csv(url,delimiter='\t',encoding='utf-8')
data.head(10)

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [0]:
data = df.sample(frac=1).reset_index(drop=True)
data.head()

In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(data ['Phrase'], data ['Sentiment'], test_size=0.3, random_state=2003)
documents=[]
X_train = np.array(X_train.values.tolist())
Y_train = np.array(Y_train.values.tolist())

for i in range(len(X_train)):
  documents.append([list(word_tokenize(X_train[i])), Y_train[i]]) 

X_test = np.array(X_test.values.tolist())
Y_test = np.array(Y_test.values.tolist())
for i in range(len(X_test)):
  documents.append([list(word_tokenize(X_test[i])), Y_test[i]]) 

documents[0]

In [0]:
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer 
porter = PorterStemmer() 
lancaster=LancasterStemmer() 
wordnet_lemmatizer = WordNetLemmatizer() 
stopwords_en = stopwords.words("english") 
punctuations="?:!.,;'\"-()"
#parameters to adjust to see the impact on outcome 
remove_stopwords = True
useStemming = True
useLemma = False
removePuncs = True


In [0]:
for l in range(len(documents)):                    
  label = documents[l][1]                          
  tmpReview = []                                   
  for w in documents[l][0]:                        
    newWord = w                                    
    if remove_stopwords and (w in stopwords_en):   
      continue                                     
    if removePuncs and (w in punctuations):        
      continue                                     
    if useStemming:
      newWord = lancaster.stem(newWord)  
    if useLemma: 
      newWord = wordnet_lemmatizer.lemmatize(newWord) 
    tmpReview.append(newWord)                      
  documents[l] = (tmpReview, label)              
  documents[l] = (' '.join(tmpReview), label) 

print(documents[0])

In [0]:
data = pd.DataFrame(documents, columns=['text', 'sentiment']) 
data.head()



In [0]:
X_train, X_test, Y_train, Y_test = train_test_split(data['text'],  data['sentiment'], test_size=0.3, random_state=2003)

In [0]:
from sklearn.feature_extraction.text import CountVectorizer , TfidfVectorizer
from keras.utils import to_categorical

vectorizer = TfidfVectorizer(max_features = 2000,ngram_range=(1, 3)) 

X_train = vectorizer.fit_transform(X_train).toarray()
# Y_train = Y_train 
X_test = vectorizer.transform(X_test).toarray()
# Y_test = Y_test
Y_train = keras.utils.to_categorical(Y_train, num_classes)
Y_test = keras.utils.to_categorical(Y_test, num_classes)

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K

In [0]:

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [0]:
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [0]:
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
batch_size = 128
num_classes = 5
epochs = 10

In [0]:
# Y_train = keras.utils.to_categorical(Y_train, num_classes)
# Y_test = keras.utils.to_categorical(Y_test, num_classes)


In [0]:
from keras.preprocessing import sequence,text
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Input, Dense, Embedding, Flatten
from keras.layers import SpatialDropout1D
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential

In [0]:
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3,
                 activation='relu',
                 input_shape=(2000,1)))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(rate = 0.5))
model.add(Flatten())
model.add(Dense(15))
model.add(Dense(num_classes, activation='softmax'))


In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.adam(),
              metrics=['accuracy',f1_m,precision_m, recall_m])  # adam optimizer from keras is called directly

In [0]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [0]:
model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs)
score = model.evaluate(X_test, Y_test, verbose=0)

In [0]:
from sklearn.metrics import classification_report, accuracy_score,precision_score,recall_score, f1_score
from keras.utils import to_categorical
output=model.predict_classes(X_test,verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('f1 score:',score[2])
print('Precision:',score[3])
print('Recall:',score[4])

In [0]:
model.save("1107159_1dconv_reg.h5")

In [0]:
model.summary()